In [1]:
#주정차단속 위치를 나타내는 데이터를 읽어온다. 위 활동을 함으로써 얻을 수 있는 효과는 견인차량보관소 최적의 위치를 머신러닝으로 찾을 수 있다.
#화재는 언제나 조심해야한다 화재에 대비하기 위해선 좁은 골목에 소방차가 들어갈 수 있어야 한다.
#따라서 효율적인 견인차량보관소의 위치를 지정하기 위해서 다음과 같은 분석을 실시하기로 하였다.
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("C:/Users/user/Desktop/주식동향예측/주정차위반단속위치현황(2018년).csv",engine='python')

In [3]:
train.head(5)

,집계년도,시군명,시군코드,관리기관명,단속일시정보,단속장소,단속방법,Unnamed: 7,Unnamed: 8
0,2018,가평군,41820,가평군,2018-06-30 16:56,가평역 삼거리 주변1,고정식CCTV,NaN,NaN
1,2018,가평군,41820,가평군,2018-06-29 18:02,자라목방향,고정식CCTV,NaN,NaN
2,2018,가평군,41820,가평군,2018-06-29 15:34,사거리방향,고정식CCTV,NaN,NaN
3,2018,가평군,41820,가평군,2018-06-29 15:25,가평역 삼거리 주변1,고정식CCTV,NaN,NaN
4,2018,가평군,41820,가평군,2018-06-29 15:06,청평면 여울길,이동식CCTV,NaN,NaN


In [4]:
train.columns

Index(['집계년도', '시군명', '시군코드', '관리기관명', '단속일시정보', '단속장소', '단속방법', 'Unnamed: 7',
       'Unnamed: 8'],
      dtype='object')

In [5]:
#필요없는 행 삭제
train = train.drop(columns=['Unnamed: 7','Unnamed: 8'])

In [6]:
# NaN값이 몇개있는지 확인
len(train.단속방법[train.단속방법.isna()])

40074

In [7]:
#단속방법에도 알기 힘든 값이 존재
train.단속방법.value_counts()

고정형CCTV            247933
주행형CCTV            123201
고정형                123168
차량이동형              112993
고정형CCTV단속           99855
PDA                 66576
PDA단속               56822
차량형CCTV             43510
이동형CCTV단속           38249
차량이동형CCTV           31615
주행형                 23014
기타(모바일)             15151
스마트폰단속              10220
수기단속                 8043
시민신고웹                4244
민원신고                 1479
이동식CCTV              1070
고정식CCTV               781
일반도보                  294
 상탑초교 맞은 편 주변          19
코너)주변                  19
민원제보                   10
 상탑초교 맞은 편              9
한진                      9
코너)                     9
코너                      7
롯데아울렛주변                 7
인력단속                    7
보도차단)                   6
소방서단속                   6
                    ...  
 진흥부동산 앞                1
밤가시마을7단지주변              1
 갈비지존(스크린골프) 주변         1
종달새공원 주변(보도차단)          1
우성 삼거리                  1
 한국디자인 진흥원 앞            1
 세계로마트 앞                1
 진흥아파트 후문   

In [8]:
#단속장소 NaN값 확인
train[train.단속장소.isna()]

,집계년도,시군명,시군코드,관리기관명,단속일시정보,단속장소,단속방법
546540,2018,성남시,41130,중원구청,2018-03-21 12:19,NaN,시민신고웹
561212,2018,성남시,41130,중원구청,2018-02-26 20:53,NaN,일반도보
564692,2018,성남시,41130,중원구청,2018-02-21 14:46,NaN,시민신고웹
892797,2018,시흥시,41390,시흥시청,2018-01-03 9:15,NaN,PDA단속
917896,2018,안산시,41270,단원구,2018-06-19 19:12,NaN,고정형CCTV단속
953733,2018,안산시,41270,단원구,2018-04-20 16:20,NaN,고정형CCTV단속
990237,2018,안산시,41270,단원구,2018-02-23 19:17,NaN,고정형CCTV단속
996255,2018,안산시,41270,단원구,2018-02-09 18:15,NaN,고정형CCTV단속
997044,2018,안산시,41270,단원구,2018-02-09 8:49,NaN,고정형CCTV단속
1009535,2018,안산시,41270,단원구,2018-01-20 17:23,NaN,고정형CCTV단속


In [9]:
#단속장소 NaN값은 제거하고 처리(행단위로 제거), 본인의 생각은 불필요하다고 판단되기 때문에...
train = train.dropna(subset=['단속장소'])

In [10]:
#NaN값이 사라진 것을 확인
train[train.단속장소.isna()].sum()

집계년도      0.0
시군명       0.0
시군코드      0.0
관리기관명     0.0
단속일시정보    0.0
단속장소      0.0
단속방법      0.0
dtype: float64

In [11]:
#각 열이 어떻게 구성되어 있는지 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048564 entries, 0 to 1048574
Data columns (total 7 columns):
집계년도      1048564 non-null int64
시군명       1048564 non-null object
시군코드      1048564 non-null int64
관리기관명     1048564 non-null object
단속일시정보    1048564 non-null object
단속장소      1048564 non-null object
단속방법      1008490 non-null object
dtypes: int64(2), object(5)
memory usage: 64.0+ MB


In [12]:
#단속장소 중 구체적인 값이 아닌 값은 제거하고 처리 ('앞'이라고만 써있는 데이터 처리)
train = train.drop(train[train.단속장소=='앞'].index)

In [13]:
#단속장소 중 구체적인 값이 아닌 값은 제거하고 처리 ('주변'이라고만 써있는 데이터 처리)
train = train.drop(train[train.단속장소=='주변'].index)

In [25]:
#몇 개의 단어와 rstrip 메소드를 이용하여 단속장소를 정제한다.
#데이터를 확인해보면 장소 마지막에 '부근', '앞', '주변', '방향' 이라는 단어가 존재하기 때문에 구글맵에서 위도,경도를 따오기 쉽지 않다.
#따라서 주변에서 발견된 내용이기 떄문에 앞의 단어를 제외한 대표 위치를 검색하여 위도,경도를 따온다.
filter_place = []
for i in range(len(train.단속장소)):
    if '(' in train.iloc[i,5]:
        filter_place.append(train.iloc[i,5][:train.iloc[i,5].index('(')].rstrip())
    elif train.iloc[i,5][-2:]=='부근':
        filter_place.append(train.iloc[i,5][:-2].rstrip())
    elif train.iloc[i,5][-1]=='앞':
        filter_place.append(train.iloc[i,5][:-1].rstrip())
    elif '주변' in train.iloc[i,5]:
        filter_place.append(train.iloc[i,5][:train.iloc[i,5].index('주변')].rstrip())
    elif train.iloc[i,5][-2:]=='방향':
        filter_place.append(train.iloc[i,5][:-2].rstrip())
    else:
        filter_place.append(train.iloc[i,5])
        

In [26]:
#정제된 데이터 확인, 확인해보면 '구역전' 이라는 값이 나온다. 이 값은 정확한 값이 아니기 때문에 후에 제거해준다.
filter_place

['가평역 삼거리',
 '자라목',
 '사거리',
 '가평역 삼거리',
 '청평면 여울길',
 '청평면 청평중앙로',
 '청평면 청평중앙로72번길',
 '가평역 삼거리',
 '자라목',
 '가평읍 석봉로191번길',
 '가평읍 문화로',
 '가평읍 연인2길',
 '현리조정면사무소',
 '자라목',
 '청평초등학교',
 '청평면 청평역로',
 '설악면사무소',
 '청평면 여울길',
 '청평면 청평중앙로',
 '청평면 청평중앙로',
 '가평읍 연인2길',
 '가평읍 연인2길',
 '북면',
 '하나로마트',
 '자라목',
 '설악터미널',
 '청평서울',
 '가평읍 보납로18번길',
 '청평면 청평중앙로72번길',
 '청평면 여울길',
 '청평면 청평중앙로',
 '청평면 청평중앙로',
 '청평면 청평중앙로',
 '조종면 조종새싹로',
 '가평읍 연인2길',
 '가평역 삼거리',
 '가평읍 연인1길',
 '청평면 청평중앙로',
 '가평읍 연인2길',
 '북면',
 '가평역 삼거리',
 '가평역 삼거리',
 '북면',
 '가평읍 보납로',
 '현리포천',
 '현리조정면사무소',
 '가평역 삼거리',
 '청평면 청평중앙로',
 '청평면 청평중앙로',
 '청평면 청평중앙로72번길',
 '북면',
 '현리조정면사무소',
 '청평농협',
 '설악면사무소',
 '청평면 청평역로',
 '청평면 청평역로',
 '청평면 청평역로',
 '청평면 청평중앙로',
 '청평면 여울길',
 '청평면 청평중앙로',
 '청평면 여울길',
 '청평면 청평중앙로',
 '청평면 청평중앙로',
 '가평읍 석봉로',
 '가평읍 석봉로',
 '현리포천',
 '가평읍 석봉로',
 '가평읍 석봉로',
 '가평읍 석봉로',
 '가평읍 석봉로',
 '가평역 삼거리',
 '설악우체국',
 '가평역 삼거리',
 '가평역 삼거리',
 '청평파출소',
 '가평읍 문화로',
 '청평면 여울길',
 '청평면 청평중앙로',
 '청평면 청평중앙로',
 '청평농협',
 '가평뚝배기짬봉앞',
 '청평면 청평중앙로26번길',
 '청

In [27]:
train['정제위반장소']=filter_place

In [28]:
train.head(5)

,집계년도,시군명,시군코드,관리기관명,단속일시정보,단속장소,단속방법,정제위반장소
0,2018,가평군,41820,가평군,2018-06-30 16:56,가평역 삼거리 주변1,고정식CCTV,가평역 삼거리
1,2018,가평군,41820,가평군,2018-06-29 18:02,자라목방향,고정식CCTV,자라목
2,2018,가평군,41820,가평군,2018-06-29 15:34,사거리방향,고정식CCTV,사거리
3,2018,가평군,41820,가평군,2018-06-29 15:25,가평역 삼거리 주변1,고정식CCTV,가평역 삼거리
4,2018,가평군,41820,가평군,2018-06-29 15:06,청평면 여울길,이동식CCTV,청평면 여울길


In [30]:
#'구역전' 이라고 표시된 데이터 제거
train = train.drop(train.정제위반장소[train.정제위반장소=='구역전'].index)

In [31]:
#데이터 제거 확인
train[train.정제위반장소=='구역전']

,집계년도,시군명,시군코드,관리기관명,단속일시정보,단속장소,단속방법,정제위반장소


In [32]:
train.to_csv('정제된단속장소포함.csv', header=False)